In [6]:
# Import required libraries
import matplotlib.pyplot as plt
#plt.rcParams['pdf.fonttype'] = 42
import pandas as pd
from IPython.core.display import HTML
import numpy as np
%matplotlib inline
import glob
import readData
import re


In [2]:
## Read in the data as a pandas dataframe
# Files should be .csv format in \Data\ subdirectory

allFiles = glob.glob("Data\*.csv") #get all files with csv extension in this subfolder
print allFiles


#Return pandas dataframe with all the data, datetime index
#toggle saveCSV True/False to save a csv of the compiled data

radius = readData.readCSV(allFiles, saveCSV = False) 
radius.tail()


['Data\\RIV_NORTH_12-29-2015-1428.csv', 'Data\\RIV_SOUTH_12-29-2015-1454.csv', 'Data\\SAGE_NORTH_2016-01-07-1204-8.csv', 'Data\\SAGE_SOUTH_2016-01-07-1307-8.csv']


,Data\SAGE_NORTH_2016-01-07-1204-8.csv0,Data\SAGE_NORTH_2016-01-07-1204-8.csv1,Data\SAGE_NORTH_2016-01-07-1204-8.csv2,Data\SAGE_NORTH_2016-01-07-1204-8.csv3,Data\SAGE_NORTH_2016-01-07-1204-8.csv4,Data\SAGE_NORTH_2016-01-07-1204-8.csv5,Data\SAGE_NORTH_2016-01-07-1204-8.csv6,Data\SAGE_NORTH_2016-01-07-1204-8.csv7,Data\SAGE_NORTH_2016-01-07-1204-8.csv8,Data\RIV_SOUTH_12-29-2015-1454.csv0,...,Data\RIV_NORTH_12-29-2015-1428.csv8,Data\SAGE_SOUTH_2016-01-07-1307-8.csv0,Data\SAGE_SOUTH_2016-01-07-1307-8.csv1,Data\SAGE_SOUTH_2016-01-07-1307-8.csv2,Data\SAGE_SOUTH_2016-01-07-1307-8.csv3,Data\SAGE_SOUTH_2016-01-07-1307-8.csv4,Data\SAGE_SOUTH_2016-01-07-1307-8.csv5,Data\SAGE_SOUTH_2016-01-07-1307-8.csv6,Data\SAGE_SOUTH_2016-01-07-1307-8.csv7,Data\SAGE_SOUTH_2016-01-07-1307-8.csv8
2016-01-08 11:00:00,0.98692,0.65795,5.9215,425.97,5.9215,483.32,-22.206,-36.681,14.15,NaN,...,NaN,685.19,1.9817,394.20,565.79,8.5876,92.481,-524.12,-243.75,14.40
2016-01-08 11:30:00,0.98686,0.65791,5.9212,426.00,5.9212,483.29,-21.985,-35.801,14.07,NaN,...,NaN,685.04,1.9816,394.29,565.76,8.5871,92.476,-510.99,-231.52,14.40
2016-01-08 12:00:00,0.93202,0.65789,5.9211,425.99,5.9211,483.61,-21.711,-36.842,14.26,NaN,...,NaN,685.13,1.9816,394.33,565.79,8.6419,92.474,-497.15,-221.94,14.36
2016-01-08 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,685.01,1.9265,394.33,565.73,8.9170,92.472,-504.03,-243.01,14.38
2016-01-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,685.06,1.8714,394.27,565.73,9.2471,92.471,-507.44,-263.87,14.40


In [8]:
## Read in the metadata as a pandas dataframe
metadata = pd.read_csv('PISTON_DENDROMETER_METADATA.csv',sep=',', dtype=str)
metadata.head()
# metadata.SERIAL = metadata.SERIAL.astype(str) #need to make sure again it's a string... ?
#Create dictionaries between sensor ID and metadata info
#well_id = metadata.set_index('SERIAL')['WELL ID'].to_dict()
well_id = dict(zip(metadata['SERIAL'], metadata['WELL ID'] )) 

,Site,Tree #,Species,Slope,Side of Tree,Sensor ID,SE Channel Data Logger,EX Channel Data Logger,Data Logger Turn On Date,Data Logger Turn-on Time,Height Above Ground (m) (+/- 1 m),Notes,DSID,Unnamed: 13,Unnamed: 14,Unnamed: 15,Notes.1
0,Sagehorn,205,QUGA,S,NE,7,2,1,8/6/2015,18:49,3,NaN,50000,NaN,NaN,NaN,We scraped away what we interpreted to be dead...
1,Sagehorn,205,QUGA,S,SW,27,4,2,8/6/2015,18:49,3,Threaded rods not srewed in all the way (yet)....,50001,NaN,NaN,NaN,We threaded rods approximately 15 cm in total ...
2,Sagehorn,206,QUGA,S,SW,6,1,1,8/6/2015,18:49,3,Threaded rods not srewed in all the way (yet)....,50002,NaN,NaN,NaN,NaN
3,Sagehorn,206,QUGA,S,E,23,3,2,8/6/2015,18:49,3,Threaded rods not srewed in all the way (yet)....,50003,NaN,NaN,NaN,NaN
4,Sagehorn,207,QUGA,S,E,43,6,3,8/6/2015,18:49,3,NaN,50004,NaN,NaN,NaN,NaN


In [9]:
#Here we assign each column name as a DSID (data stream ID)
#To do so, the 
for columns in radius:
    print columns
    #if column contains sage
    #    if column contains north
    #metadata where site = sage and slope = north
    #zip dictionary between SE and DSID
    #get digit from column... rename column with dsid
    
    
    #    if column contains south
    #if column contains riv
    

Data\SAGE_NORTH_2016-01-07-1204-8.csv0
Data\SAGE_NORTH_2016-01-07-1204-8.csv1
Data\SAGE_NORTH_2016-01-07-1204-8.csv2
Data\SAGE_NORTH_2016-01-07-1204-8.csv3
Data\SAGE_NORTH_2016-01-07-1204-8.csv4
Data\SAGE_NORTH_2016-01-07-1204-8.csv5
Data\SAGE_NORTH_2016-01-07-1204-8.csv6
Data\SAGE_NORTH_2016-01-07-1204-8.csv7
Data\SAGE_NORTH_2016-01-07-1204-8.csv8
Data\RIV_SOUTH_12-29-2015-1454.csv0
Data\RIV_SOUTH_12-29-2015-1454.csv1
Data\RIV_SOUTH_12-29-2015-1454.csv2
Data\RIV_SOUTH_12-29-2015-1454.csv3
Data\RIV_SOUTH_12-29-2015-1454.csv4
Data\RIV_SOUTH_12-29-2015-1454.csv5
Data\RIV_SOUTH_12-29-2015-1454.csv6
Data\RIV_SOUTH_12-29-2015-1454.csv7
Data\RIV_SOUTH_12-29-2015-1454.csv8
Data\RIV_NORTH_12-29-2015-1428.csv0
Data\RIV_NORTH_12-29-2015-1428.csv1
Data\RIV_NORTH_12-29-2015-1428.csv2
Data\RIV_NORTH_12-29-2015-1428.csv3
Data\RIV_NORTH_12-29-2015-1428.csv4
Data\RIV_NORTH_12-29-2015-1428.csv5
Data\RIV_NORTH_12-29-2015-1428.csv6
Data\RIV_NORTH_12-29-2015-1428.csv7
Data\RIV_NORTH_12-29-2015-1428.csv8
D

In [3]:
  
# voltage to microns
#radius['8']=4*radius['8']
        

In [4]:
#%% SPECIFY TIME RANGE OF SERIES
startDateTime = pd.to_datetime('2015-08-13 00:00:00')
stopDateTime = pd.to_datetime('2016-01-01 00:00:00')
radius = radius[(pd.to_datetime(radius.index) > startDateTime) & 
                (pd.to_datetime(radius.index) < stopDateTime)]


In [5]:
for i in range(4,12):
    plt.figure()
    plt.plot(radius.index, radius[i])
    plt.ylim( [min(radius[i]),max(radius[i])] )


KeyError: 4

In [ ]:
range(4,10)